In [ ]:
import nltk
from numpy import zeros, int8, log
from pylab import random
import sys
import codecs
from nltk.corpus import stopwords
nltk.download('stopwords')
import jieba
import re
import time

In [ ]:
# segmentation, stopwords filtering and document-word matrix generating
# [return]:
# N : number of documents
# M : length of dictionary
# word2id : a map mapping terms to their corresponding ids
# id2word : a map mapping ids to terms
# X : document-word matrix, N*M, each line is the number of terms that show up in the document
def preprocessing(datasetFilePath):

      #stopwords
    stops = set(stopwords.words("english"))
    # read the documents
    file = codecs.open(datasetFilePath, 'r', 'utf-8')
    documents = [document.strip() for document in file]
    file.close()

    # number of documents
    N = len(documents)

    wordCounts = [];
    word2id = {}
    id2word = {}
    currentId = 0;

    for document in documents:
        segList = jieba.cut(document)
        wordCount = {}
        for word in segList:
            word = word.lower().strip()
            if len(word) > 1 and not re.search('[0-9]', word) and word not in stops:
                if word not in word2id.keys():
                    word2id[word] = currentId;
                    id2word[currentId] = word;
                    currentId += 1;
                if word in wordCount:
                    wordCount[word] += 1
                else:
                    wordCount[word] = 1
        wordCounts.append(wordCount);

    # length of dictionary
    M = len(word2id)

    # generate the document-word matrix
    X = zeros([N, M], int8)
    for word in word2id.keys():
        j = word2id[word]
        for i in range(0, N):
            if word in wordCounts[i]:
                X[i, j] = wordCounts[i][word];


    return N, M, word2id, id2word, X


def initializeParameters():
    for i in range(0, N):
        normalization = sum(PI[i, :])
        for j in range(0, K):
            PI[i, j] /= normalization;

    for i in range(0, K):
        normalization = sum(theta[i, :])
        for j in range(0, M):
            theta[i, j] /= normalization;


def EStep():
    for i in range(0, N):
        for j in range(0, M):
            denominator = 0
            for k in range(0, K):
                p[i, j, k] = theta[k, j] * PI[i, k]
                denominator += theta[k, j] * PI[i, k]
            for k in range(0, K):
                p[i, j, k] /= float(denominator)

def MStep():
    for k in range(0, K):
        denominator = 0
        for j in range(0, M):
            for i in range(0, N):
                theta[k, j] += p[i, j, k] * X[i, j]
                denominator += p[i, j, k] * X[i, j]
        for j in range(0, M):
            theta[k, j] /= float(denominator)
    for i in range(0, N):
        denominator = 0
        for k in range(0, K):
            for j in range(0, M):
                PI[i, k] += p[i, j, k] * X[i, j]
                denominator += p[i, j, k] * X[i, j]
        for k in range(0, K):
            PI[i, k] /= float(denominator)


# calculate the log likelihood
def LogLikelihood():
    loglikelihood = 0
    for i in range(0, N):
        loglikelihood += log(1/float(N))
        for j in range(0, M):
            p = 0
            for k in range(0, K):
                p += PI[i, k] * theta[k, j]
            loglikelihood += log(p) * X[i, j]
    return loglikelihood


# calculate the log likelihood with Background
def LogLikelihoodBG(lamb):

    WordCount = sum(sum(X))

    BG = zeros([M], int8)
    for i in range(0,M):
      wordInDocs = sum(X.T[i])
      BG[i]=wordInDocs/WordCount

    loglikelihood = 0
    for i in range(0, N):
        loglikelihood += log(1/float(N))
        for j in range(0, M):
            p = BG[j]*lamb
            for k in range(0, K):
                p += (PI[i, k] * theta[k, j]) * (1 - lamb)
            loglikelihood += log(p) * X[i, j]
    return loglikelihood


# output the params of model and top words of topics to files
def output():
    # document-topic distribution
    file = codecs.open(docTopicDist,'w','utf-8')
    for i in range(0, N):
        tmp = ''
        for j in range(0, K):
            tmp += str(PI[i, j]) + ' '
        file.write(tmp + '\n')
    file.close()

    # topic-word distribution
    file = codecs.open(topicWordDist,'w','utf-8')
    for i in range(0, K):
        tmp = ''
        for j in range(0, M):
            tmp += str(theta[i, j]) + ' '
        file.write(tmp + '\n')
    file.close()

    # dictionary
    file = codecs.open(dictionary,'w','utf-8')
    for i in range(0, M):
        file.write(id2word[i] + '\n')
    file.close()

    # top words of each topic
    file = codecs.open(topicWords,'w','utf-8')
    for i in range(0, K):
        topicword = []
        ids = theta[i, :].argsort()
        for j in ids:
            topicword.insert(0, id2word[j])
        tmp = ''
        for word in topicword[0:min(topicWordsNum, len(topicword))]:
            tmp += word + ' '
        file.write(tmp + '\n')
    file.close()

In [ ]:
# set the default params and read the params from cmd
datasetFilePath = "dataset.txt"
K = 10   # number of topic
maxIteration = 30
threshold = 0.0001
topicWordsNum = 10
docTopicDist = "docTopicDist.txt"
topicWordDist = 'topicWordDistribution.txt'
dictionary = 'dictionary.dic'
topicWords = "topics.txt"

# preprocessing
N, M, word2id, id2word, X = preprocessing(datasetFilePath)

# theta[i, j] : p(zj|di): 2-D matrix
PI = random([N, K])
# beta[i, j] : p(wj|zi): 2-D matrix
theta = random([K, M])
# p[i, j, k] : p(zk|di,wj): 3-D tensor
p = zeros([N, M, K])

initializeParameters() # normarlize

# EM algorithm
oldLoglikelihood = 1
newLoglikelihood = 1
for i in range(0, maxIteration):
    EStep() #implement E step
    MStep() #implement M step
    # newLoglikelihood = LogLikelihood()
    newLoglikelihood = LogLikelihoodBG(0.3)
    print("[", time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())), "] ", i+1, " iteration  ", str(newLoglikelihood))
    # you should see increasing loglikelihood
    if(oldLoglikelihood != 1 and newLoglikelihood - oldLoglikelihood < threshold):
        break
    oldLoglikelihood = newLoglikelihood

output()